NATURAL LANGUAGE PROCESSING FOR PRODUCT TITLES OF SHOPPEE

- NAME- Surbhi Prasad, Karishma Chauhan
- GOAL= Find Similar Products from title Information
- ALGORITHM- LogisticRegression


<b> RUNTIME Total </b> :
- Total - ~2 hours
- Cosine Similarity- 10 mins (Reduced time from 48 mins to 10 mins using Cache)
- Classification Model - 1.92 hours

<b> AIM </b> : 
Find closest products for Shoppee website based on data provided by ecommerce. This data has ~32.5K images and its title. Idea is to utilise title and descriptions of products in predicting closest pther products for recommendation.

<b> METHOD </b> :

1. Data Import: Loaded process text file from MongoDB for all images with titles.
2. Label Group Encoding: Encoded label code(String Indexer) which is product category of image.
3. Paired Dataset: Created all possible pairs of titles based on rule that atleast there are 5 sets of titles for having enough dataset for training for each label group.
4. Target variable Creation(for Binary Classification): Pairs of titles of same label group were classified as 1 and remaining pairs of non equal label group as 0.
5. Negative Sampling to Balance Data: Since, we have ~100 times 0s than 1s, we sampled zeros from data. We kept equal samples of negative class as positive samples in data for 1:1 ratio.
6. Embeddings Spark NLP Pipeline: Extracted two multilingual embeddings as most of the products are of Indonesian market.Extracted last layer of Robeta Bert (of 768 dimemsional/features veactor) and UniversalWord(500 feature vector) embeddings from Spark Package in TensorFlow.
6. Embeddings Cosine Similarity: Calulcated distance(d) for each paired title embeddings for both embeddings to be fed in later Classification model.
7. Train-Test Split: Created Training data and validation data based on Random split of 0.8/0.2 and chosed metric as 'accuracy' for tuning
8. Classification Pipeline: 
a) Utilized Cosine distances(d) as "features" and Binary Label as "labels" for Classification.
b) Modified data based on vector Assembler and ran Logistic Regression Pipeline and Random Forest pipeline
c) Ran 5 fold Cross validation with logistic regression with improvement on validation accuracy to 0.74
9) Ensembled both model results.
10) Evaluated on Test Data.

<b> RESULT </b>

Accuracy Achieved: 
- Train: 0.659
 - Val: 0.657

Cluster: 
- Group-3 GPU- 64GB,16Cores,DBR 10.3 ML,Spark 3.21
- It was used since embeddings were computationally expensive to run(~800 features)

Main Packages Used: 
- Spark NLP version:  3.4.2
- Apache Spark version:  3.2.1

Future Scope:
1. Tune Bert Embeddings based on Argface loss method for model accuracy improved.
2. Create Ensemble Model of two classification methods for final prediction.
3. Can utilise this model for combination with Image models for ensemble.

In [0]:
!pip install sparknlp #this sometimes not work if not added in maven "johnsnowlabs spark NLP"


In [0]:
import sparknlp

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as sqlF
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
import tensorflow as tf
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql.functions import shuffle,rand
import torch
from pyspark.sql.functions import split, col
from pyspark.sql.functions import explode
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

Out[3]: device(type='cuda', index=0)

In [0]:
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","16G")\
    .config('spark.executor.memory ',
            
            
            '16G')\
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.network.timeout", "36000000s")\
    .config("spark.executor.heartbeatInterval", "3600s")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.1")\
    .getOrCreate()

In [0]:
spark.version

Out[5]: '3.2.1'

In [0]:
import sparknlp
spark = sparknlp.start(spark32=True)

In [0]:
database = 'Group3'
collection = 'train_csv'
user_name = 'root'
password = 'root'
address = 'cluster0.eq07a.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
df=spark.read.format("mongo").option("uri",connection_string).load().cache()

LOAD DATA PROCSEED FILE FROM MONGODBFOR TRAINING MODEL

In [0]:

df.printSchema()

root
-- _id: struct (nullable = true)
 |-- oid: string (nullable = true)
-- image: string (nullable = true)
-- image_phash: string (nullable = true)
-- label_group: string (nullable = true)
-- posting_id: string (nullable = true)
-- title: string (nullable = true)

LABEL ENCODING FOR NON CONTINUOUS LABLE GROUP IDS FOR BETTER USAGE IF CONTINUOUS CATEGORIES

In [0]:
indexer = StringIndexer(inputCol="label_group", outputCol="label_code") 
indexed = indexer.setHandleInvalid("skip").fit(df).transform(df) 
indexed.show()

+--------------------+--------------------+----------------+-----------+----------------+--------------------+----------+
 _id| image| image_phash|label_group| posting_id| title|label_code|
+--------------------+--------------------+----------------+-----------+----------------+--------------------+----------+
{622d63de38c52f0f...|e23f62ae15ae35b15...|9f1b58f8e2900fac| 504302079|train_1698340041|OTIN FASHION baj...| 10047.0|
{622d63de38c52f0f...|e242004183bb95674...|b0c7cb34cc1ccc73| 153145669|train_3206735432|Ernest Tote Mini ...| 4929.0|
{622d63de38c52f0f...|e242f927f3f10c73d...|c3fda103de0e4c3c| 1099725405|train_2666269465|Lianhua Qingwen J...| 157.0|
{622d63de38c52f0f...|e244b0d42306a47b4...|be91b194b4966969| 3460499189|train_1688611703|PEDIASURE COMPLET...| 1926.0|
{622d63de38c52f0f...|e24c61bc81049a21f...|fae18d16e18e94e2| 524272944| train_177136644|Hanasui Anti Agin...| 891.0|
{622d63de38c52f0f...|e24d38babbe3ef9e5...|ee6295c95a9acc26| 2378308641| train_556509614| RC Paket Bundle A| 6510.0|
{622d63de38c52f0f...|e24d3fa0fb6fd2bb5...|be80d534417f4be1| 2403542018|train_1926162836|Mata Bor Set Kenm...| 6563.0|
{622d63de38c52f0f...|e25176686fa13462d...|eb7097878cdec058| 849367919|train_1605223031|Handy Stitch/ Por...| 3952.0|
{622d63de38c52f0f...|e2523aa70c94df469...|e99c34c69993c6c6| 3488861486|train_1026366300|Polytex spon serb...| 1940.0|
{622d63de38c52f0f...|e258f4f451bf5adba...|e96b9669c518c5cc| 1634389569|train_1576267703|Belfoods Uenaaak ...| 5105.0|
{622d63de38c52f0f...|e2593d7a92f050a84...|ebdae5858cd0948d| 2441397072|train_1106529266|Jasmineolshop KEM...| 6635.0|
{622d63de38c52f0f...|e25c680023c2b948f...|935932c6645ce3b3| 2401169789|train_2191928997|opp plastik/plast...| 315.0|
{622d63de38c52f0f...|e25deaa4cf2710eb3...|d43d43782a7c73a4| 1688029306|train_1175574739|Ikat Rambut Bahan...| 2563.0|
{622d63de38c52f0f...|e25fc50ce57ec6fc6...|c90eb2e08f9fe05a| 4148428209|train_2594604400|Brownie Chocolate...| 9673.0|
{622d63de38c52f0f...|e2618994d5701c013...|aec9d91dd262251b| 1514327608| train_115217781|Bantal Pijat spee...| 77.0|
{622d63de38c52f0f...|e2630fd5b4dff7b4f...|fb53e12e1fa05984| 21545740|train_1763260210|Top Seller\xe2\x9...| 2773.0|
{622d63de38c52f0f...|e2653642f51285fd7...|ee8bcb14f465914c| 1044028636|train_2530782075|IORA collection -...| 1407.0|
{622d63de38c52f0f...|e2653642f51285fd7...|ee8bcb14f465914c| 1044028636|train_3114669608|OTIN FASHION baju...| 1407.0|
{622d63de38c52f0f...|e2653642f51285fd7...|ee8bcb14f465914c| 1044028636|train_1107062772|kasih fashion jak...| 1407.0|
{622d63de38c52f0f...|e265c44f05699f90e...|ade48c96f1a0e3e2| 3741723479|train_2833966941|Clip On Mic Micro...| 122.0|
+--------------------+--------------------+----------------+-----------+----------------+--------------------+----------+
only showing top 20 rows

FILTERED DATA BASED ON ATLEAST 5 IMAGES FOR TRAINING IN A LABEL GROUP

In [0]:
df_gt_4_cnt=indexed.groupby("label_code").count().filter("count>5")

In [0]:
df_gt_4=indexed.filter(f.col("label_code").isin(list(np.array(df_gt_4_cnt.select('label_code').collect()).reshape(-1))))

In [0]:
df_filtered= df_gt_4.select('title','image','label_code').crossJoin(df_gt_4.select('title','image','label_code')).orderBy(rand())
newcolumns = ['title1','image1',
 'label_code1',
 'title2',           
 'image2',
 'label_code2',]
result_df=df_filtered.toDF(*newcolumns)

CREATED BINARY LABEL FOR ALL POSSIBLE PAIRS OF PRODUCTS.

In [0]:
result_df_label=result_df.withColumn('binary_label',f.when((result_df.label_code1== result_df.label_code2), '1').otherwise('0') )

In [0]:
result_df_label=result_df_label.filter("image1!=image2")

In [0]:
cnt_1=(result_df_label.filter("binary_label==1").count())/2

In [0]:
neg_frac=cnt_1/(result_df_label.filter("binary_label==0").count())

In [0]:
df_0=result_df_label.filter("binary_label==0").sample(fraction=neg_frac,seed=3)

In [0]:
df_total=df_0.union(result_df_label.filter("binary_label==1"))

In [0]:
df_total.printSchema()

root
-- title1: string (nullable = true)
-- image1: string (nullable = true)
-- label_code1: double (nullable = false)
-- title2: string (nullable = true)
-- image2: string (nullable = true)
-- label_code2: double (nullable = false)
-- binary_label: string (nullable = false)

In [0]:
df_total.cache()

Out[22]: DataFrame[title1: string, image1: string, label_code1: double, title2: string, image2: string, label_code2: double, binary_label: string]

In [0]:
df_total.printSchema()

root
-- title1: string (nullable = true)
-- image1: string (nullable = true)
-- label_code1: double (nullable = false)
-- title2: string (nullable = true)
-- image2: string (nullable = true)
-- label_code2: double (nullable = false)
-- binary_label: string (nullable = false)

In [0]:
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version: 3.4.2
Apache Spark version: 3.2.1

In [0]:
!java -version

openjdk version "1.8.0_302"
OpenJDK Runtime Environment (Zulu 8.56.0.21-CA-linux64) (build 1.8.0_302-b08)
OpenJDK 64-Bit Server VM (Zulu 8.56.0.21-CA-linux64) (build 25.302-b08, mixed mode)

Extract embeddings for all pairs using SPARK NLP pipeline

In [0]:
document = DocumentAssembler()\
    .setInputCol("title1")\
    .setOutputCol("document")
embeddings = XlmRoBertaSentenceEmbeddings.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
classsifierdl = MultiClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("label_code1")\
    .setBatchSize(128) \
    .setMaxEpochs(10) \
    .setLr(1e-3) \
    .setValidationSplit(0.1)
embeddingsFinisher = EmbeddingsFinisher()\
  .setInputCols("sentence_embeddings")\
  .setOutputCols("finished_embeddings")\
  .setOutputAsVector(True)
pipeline = Pipeline().setStages([
    document,
    embeddings,
    embeddingsFinisher
])

sent_xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
pipelineModel = pipeline.fit(df_total)

In [0]:
result = pipelineModel.transform(df_total)

In [0]:
result.cache()

Out[29]: DataFrame[title1: string, image1: string, label_code1: double, title2: string, image2: string, label_code2: double, binary_label: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence_embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, finished_embeddings: array<vector>]

In [0]:
np.array(result.select("finished_embeddings").limit(1).collect()).shape

Out[32]: (1, 1, 1, 768)

In [0]:

result_new=result.select(result.title1,result.image1,result.label_code1,result.title2,result.image2,result.label_code2,result.binary_label,explode(result.finished_embeddings))

In [0]:
result_new.cache()

Out[31]: DataFrame[title1: string, image1: string, label_code1: double, title2: string, image2: string, label_code2: double, binary_label: string, col: vector]

In [0]:
document = DocumentAssembler()\
    .setInputCol("title2")\
    .setOutputCol("document")
embeddings = XlmRoBertaSentenceEmbeddings.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
classsifierdl = MultiClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("label_code2")\
    .setBatchSize(128) \
    .setMaxEpochs(10) \
    .setLr(1e-3) \
    .setValidationSplit(0.1)
embeddingsFinisher = EmbeddingsFinisher()\
  .setInputCols("sentence_embeddings")\
  .setOutputCols("finished_embeddings")\
  .setOutputAsVector(True)
pipeline = Pipeline().setStages([
    document,
    embeddings,
    embeddingsFinisher
])

sent_xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(result_new)
result = pipelineModel.transform(result_new)

In [0]:

result_new=result.select(result.title1,result.image1,result.label_code1,result.title2,result.image2,result.label_code2,result.binary_label,result.col,explode(result.finished_embeddings))

In [0]:
result_new.cache()

Out[31]: DataFrame[title1: string, image1: string, label_code1: double, title2: string, image2: string, label_code2: double, binary_label: string, col: vector, col: vector]

In [0]:
result_new.printSchema()

root
-- title1: string (nullable = true)
-- image1: string (nullable = true)
-- label_code1: double (nullable = false)
-- title2: string (nullable = true)
-- image2: string (nullable = true)
-- label_code2: double (nullable = false)
-- binary_label: string (nullable = false)
-- col: vector (nullable = true)
-- col: vector (nullable = true)

In [0]:
newcolumns = [ 'title1', 
              'image1',
              'label_code1',
              'title2' ,    
              'image2',
              'label_code2',
              'binary_label', 
              'embed1',      
              'embed2']
result_new=result_new.toDF(*newcolumns)

In [0]:
result_new.printSchema()

root
-- title1: string (nullable = true)
-- image1: string (nullable = true)
-- label_code1: double (nullable = false)
-- title2: string (nullable = true)
-- image2: string (nullable = true)
-- label_code2: double (nullable = false)
-- binary_label: string (nullable = false)
-- embed1: vector (nullable = true)
-- embed2: vector (nullable = true)

In [0]:
result_df_add.filter('binary_label == 0').count()

In [0]:
result_new.cache()

Out[36]: DataFrame[title1: string, image1: string, label_code1: double, title2: string, image2: string, label_code2: double, binary_label: string, embed1: vector, embed2: vector]

Cosine Similarity Calculation

In [0]:
import math

def cosine_similarity(X, Y):
    den = X.norm(2) * Y.norm(2)
    if den == 0.0:
        return -1.0
    else:
        return X.dot(Y) / float(den)

In [0]:
result_new.printSchema()

root
-- title1: string (nullable = true)
-- image1: string (nullable = true)
-- label_code1: double (nullable = false)
-- title2: string (nullable = true)
-- image2: string (nullable = true)
-- label_code2: double (nullable = false)
-- binary_label: string (nullable = false)
-- embed1: vector (nullable = true)
-- embed2: vector (nullable = true)

In [0]:
pairProdDF = result_new.rdd.map(lambda x: (x[6],cosine_similarity(x[7], x[8])))


In [0]:
def toDoubleSafe(v):
    try:
        return float(v)
    except ValueError:
        return 0.0

In [0]:
pairProdDF_rdd = pairProdDF.map(lambda x: (x[0],toDoubleSafe(x[1])))

In [0]:
pairProdDF_DF=pairProdDF_rdd.toDF()

In [0]:
pairProdDF_DF.cache()

Out[44]: DataFrame[_1: string, _2: double]

Classification Model

In [0]:

input_cols=["_2"]

va = VectorAssembler(outputCol="features", inputCols=input_cols)
#lpoints - labeled data.
lpoints = va.transform(pairProdDF_DF).select("features", "_1").withColumnRenamed("_1", "label")

In [0]:
vlpoints=lpoints.sample(fraction=0.005,seed=3)

In [0]:
vlpoints.cache()

Out[55]: DataFrame[features: vector, label: int]

In [0]:
vlpoints

In [0]:
lpoints.show(2)

+--------------------+-----+
 features|label|
+--------------------+-----+
[0.9849234186033998]| 0|
[0.9870864926248505]| 0|
+--------------------+-----+
only showing top 2 rows

In [0]:
lpoints = lpoints.withColumn("label_int",col("label").cast(IntegerType())).drop('label').withColumnRenamed("label_int","label")

In [0]:
pendtsets = lpoints.randomSplit([0.8, 0.2], 1)
pendttrain = pendtsets[0].cache()
pendtvalid = pendtsets[1].cache()

In [0]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(regParam=0.01, maxIter=10, fitIntercept=True)
lrmodel = lr.fit(pendttrain)

In [0]:
lrmodel_predict=lrmodel.transform(pendttrain)

In [0]:
train_accuracy=BinaryClassificationEvaluator().evaluate(lrmodel.transform(pendttrain))

In [0]:
BinaryClassificationEvaluator().evaluate(lrmodel.transform(pendttrain))

Out[59]: 0.6593906038899757

In [0]:
BinaryClassificationEvaluator().evaluate(lrmodel.transform(pendtvalid))

Out[52]: 0.656364515242193